In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This notebook was authored with assistance from [Ivan Nardini](https://github.com/inardini)

# Get started with Vertex AI Experiments

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexperiments%2Fget_started_with_vertex_experiments.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview


This tutorial covers getting started with Vertex AI Experiments, and demonstrates how to use Vertex AI in production.

Learn more about [Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments), [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata) and [Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training).

### Objective

In this tutorial, you learn how to use Vertex AI Experiments when training with Vertex AI.

This tutorial uses the following Google Cloud ML services:

- Vertex AI Experiments
- Vertex ML Metadata
- Vertex AI Training

The steps performed include:

- Local (notebook) training
    - Create an experiment.
    - Create a first run in the experiment.
    - Log parameters and metrics.
    - Create artifact lineage.
    - Visualize the experiment results.
    - Execute a second run.
    - Compare the two runs in the experiment.
- Cloud (Vertex AI) training
    - Within the training script:
        - Create an experiment.
        - Log parameters and metrics.
        - Create artifact lineage.
    - Create a Vertex AI training custom job.
    - Execute the custom job.
    - Visualize the experiment results,

### Recommendations

When doing E2E MLOps on Google Cloud, the following are some of the best practices for logging data when experimenting or formally training a model.

#### Python Logging

Use Python's logging package when doing ad-hoc training locally.

#### Cloud Logging

Use Cloud Logging when doing training on the cloud.

#### Experiments

Use Vertex AI Experiments in conjunction with logging when performing experiments to compare results for different experiment configurations.

### Dataset

This tutorial doesn't use a dataset. References to example datasets is for demonstration purposes only.

### Costs
This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "projet-ia-448520" # "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = "gs://cours1bucket" # f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $LOCATION $BUCKET_URI

#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

In [ ]:
SERVICE_ACCOUNT = "837843744498-compute@developer.gserviceaccount.com" # "[your-service-account]"  # @param {type:"string"}

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries

In [ ]:
import os
import uuid

import google.cloud.aiplatform as aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Introduction to Vertex AI Experiments

With Vertex AI Experiments you can log and track the following when experimenting and developing your model architecture and model training:

- Log the metaparameters for the model architecture.
- Log the hyperparameters for training.
- Log the evaluation metrics.
- Create an artifact lineage of the dataset, model and evaluation.
- Group one or more training runs under an experiment.
- Compare experiments.

Vertex AI Experiments can be integrated with the following development process flows:

- Local development in a notebook
- Cloud development in Vertex AI Training
- Operationalizing development in Vertex AI Pipelines

Learn more about [Experiments]( https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments).

Learn more about [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction).

### Local development in a notebook

You can track an experiment in your local development, such as in a Vertex AI Workbench notebook, by:

- Wrap (preamble) the creation of an experiment.
- Instantiate a run per training run in the experiment.
- Within the local training run, log the corresponding parameters and results.
- Create lineage to the artifacts and experiment data.
- Retrieve the experiment data.

#### Create experiment for tracking training related metadata

First, you create an experiment using the `init()` method and then initialize a run within the experiment using `start_run()`.

- `aiplatform.init()` - Create an experiment instance
- `aiplatform.start_run()` - Track a specific run within the experiment.

In [ ]:
# Specify a name for the experiment
your_name = ...
EXPERIMENT_NAME = f"supaero-vertex-course-1-2_{your_name}"


In [ ]:
# Create experiment
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

metaparams = {}
metaparams["units"] = 128
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)#### Log parameters for the experiment

Typically, an experiment is associated with a specific dataset and a model architecture. Within an experiment, you may have multiple training runs,
where each run involves a different configuration. For example:

- Data feeding, such as:
    - dataset split
    - dataset sampling and boosting
- Metaparameters, such as:
    - depth and width of layers
- Hyperparameters, such as:
    - batch size
    - learning rate

These configuration settings are referred to as parameters, which you store as key-value pairs using the method `log_params()`

In [ ]:
metaparams = {}
metaparams["units"] = 128
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)

#### Log metrics for the experiment

At the completion or termination of a run within an experiment, you can log results that you use to compare runs. For example:

- evaluation metrics
- hyperparameter search selection
- time to train the model
- early stop trigger

These results are referred to as metrics, which you store as key-value pairs using the method `log_metrics()`

In [ ]:
metrics = {}
metrics["test_acc"] = 93.7
metrics["train_acc"] = 97.3
aiplatform.log_metrics(metrics)

#### Get the experiment results

When you are finished with a run within an experiment, call `end_run()` method to complete the logging for that run.

Next, use the experiment name as a parameter to the method `get_experiment_df()` to get the results of the experiment as a pandas dataframe.

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

#### Start subsequent run in an experiment

Next, create a second run for the same experiment. In this example, you change the metaparameter for `units` from 126 to 256, and log different metric results.

Change other parameters in order to improve a hyopthetical training.

Change Accuracy in order to avoid overtfitting

In [ ]:
aiplatform.start_run("run-2")

metaparams = {}
metaparams["units"] = ...  # changed the value
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = ...
hyperparams["batch_size"] = ...
hyperparams["learning_rate"] = ...
aiplatform.log_params(hyperparams)

metrics = {}
metrics["test_acc"] = ...
metrics["train_acc"] = ...
aiplatform.log_metrics(metrics)

#### Comparing runs in the same experiment

Finally, use the experiment name as a parameter to the method `get_experiment_df()` to get the results of all the runs within the experiment as a pandas dataframe.

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

#### Delete the experiment

Next, delete the experiment using the `delete()` method.

In [ ]:
exp = aiplatform.Experiment(EXPERIMENT_NAME)
try:
    exp.delete()
except Exception as e:
    print(e)

## Your Turn

Find the best run for experiment supaero-vertex-course-1-1 with aiplatform module